In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K
import time

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\toolkit

In [2]:
print(K.image_data_format())

channels_last


In [3]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [4]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [5]:
def DataSet(iFold,noise_tag,dup_tag,rand_tag):
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    num_class = 2
    
    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        if (int)(dup_tag):
            num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        else:
            num_dup[index] = 1
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, num_feature), dtype=float)
    Y_train = np.empty((nrow_train_dup, num_class), dtype=float)

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(num_feature):
                X_train[sam_count,i] = float(data_train[index,i])
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, num_feature), dtype=float)
    Y_test  = np.empty((nrow_test, num_class), dtype=float)

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(num_feature):
            X_test[index,i] = float(data_test[index,i])
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_test[index] = tmp_v
        
    if (int)(noise_tag):
        noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
        output = X_train + noise
        X_train = np.concatenate((X_train, output), axis=0)
        Y_train = np.concatenate((Y_train, Y_train), axis=0)
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    #X_train_new = np.empty((nrow_train_dup, 224, 224, 3))
    #Y_train_new = Y_train
    
    #for index in range(nrow_train_dup):
    #    tmp_new = ndarray_nearest_neighbour_scaling(X_train[index],224,224)
    #    for c in range(3):
    #        X_train_new[index,:,:,c] = tmp_new[:,:,0]
    
    #X_test_new = np.empty((nrow_test, 224, 224, 3))
    #Y_test_new = Y_test
    
    #for index in range(nrow_test):
    #    tmp_new = ndarray_nearest_neighbour_scaling(X_test[index],224,224)
    #    for c in range(3):
    #        X_test_new[index,:,:,c] = tmp_new[:,:,0]
            
    #del X_train, X_test
    #gc.collect()

    return X_train,Y_train,X_test,Y_test

In [6]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [7]:
def neural(channel,length,classes):
    input_shape = (channel,length)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (length,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    #model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    #model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    #model.add(Flatten())
    #model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(600,activation=LeakyRelu,input_dim=length,name="fc1"))
    #model.add(Dense(600,activation=LeakyRelu,name="fc2"))
    #model.add(Dense(768,activation=LeakyRelu,name="fc3"))
    model.add(Dense(192,activation=LeakyRelu,name="fc4"))
    model.add(Dense(classes,activation="softmax",name="fc5"))
    return model


In [8]:
def get_statistics(model,sam_data,lab_data):
    num_class = 2

    #
    test_rows = len(lab_data)
    test_prc = np.zeros(num_class)
    test_rec = np.zeros(num_class)
    test_Fva = np.zeros(num_class)
    test_BER = np.zeros(num_class)
    tmp_beta = 1
    yhat_label = model.predict_classes(sam_data, batch_size=32, verbose=1)
    true_labels = np.zeros(len(lab_data))
    for index in range(len(lab_data)):
        tmp_vec = lab_data[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind

    test_N_TP = np.zeros(num_class)
    test_N_FP = np.zeros(num_class)
    test_N_TN = np.zeros(num_class)
    test_N_FN = np.zeros(num_class)
    tmp_count = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    test_N_TP[j]=test_N_TP[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_FP[j]=test_N_FP[j]+1
            else:
                if j == true_label:
                    test_N_FN[j]=test_N_FN[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_TN[j]=test_N_TN[j]+1

    for i in range(num_class):
        if test_N_TP[i] > 0:
            test_prc[i] = test_N_TP[i] / (test_N_TP[i] + test_N_FP[i])
        else:
            test_prc[i] = 0

        test_rec[i]= test_N_TP[i] / (test_N_TP[i] + test_N_FN[i])
        test_Fva[i]=(1 + tmp_beta*tmp_beta)*test_rec[i]*test_prc[i]/\
                      (tmp_beta*tmp_beta*(test_rec[i]+test_prc[i]))
        test_BER[i]= test_N_FN[i] / (test_N_TP[i] + test_N_FN[i])
        
    print(tmp_count)
        
    return test_N_TP,test_N_FP,test_N_TN,test_N_FN,tmp_count
    

In [9]:
all_TP_train=[]
all_FP_train=[]
all_TN_train=[]
all_FN_train=[]
all_count_train=[]
all_TP_test=[]
all_FP_test=[]
all_TN_test=[]
all_FN_test=[]
all_count_test=[]
all_time_train=[]
all_time_predict_train=[]
all_time_predict_test=[]

nRun = 10
side_len = 22
num_class = 2
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for fold in range(nRun):
                iFold = fold + 1
                X_train,Y_train,X_test,Y_test = DataSet(iFold,noise_tag,dup_tag,ran_tag)
                print('X_train shape : ',X_train.shape)
                print('Y_train shape : ',Y_train.shape)
                print('X_test shape : ',X_test.shape)
                print('Y_test shape : ',Y_test.shape)
                #print(X_train[0])
                #print(Y_train[0,1])
                print(noise_tag,dup_tag,ran_tag,iFold)
                
                num_feature = X_train.shape[1]

                # # model
                #model = ResNet50(
                #    weights=None,
                #    classes=2
                #)

                ##
                model = neural(1,num_feature,num_class)

                model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

                # # train
                start = time.clock()
                model.fit(X_train, Y_train, epochs=10, batch_size=16)
                all_time_train.append(time.clock()-start)

                # # evaluate
                #model.evaluate(X_test, Y_test, batch_size=32)

                # #
                #models.append(model)

                # # save
                #save_path =("./my_resnet_model.h5_F%d") % (iFold)
                #model.save(save_path)

                #
                X_train,Y_train,X_test,Y_test = DataSet(iFold,0,0,ran_tag)
                start = time.clock()
                train_N_TP,train_N_FP,train_N_TN,train_N_FN,train_count = get_statistics(model,X_train,Y_train)
                all_time_predict_train.append(time.clock()-start)
                start = time.clock()
                test_N_TP,test_N_FP,test_N_TN,test_N_FN,test_count = get_statistics(model,X_test,Y_test)
                all_time_predict_test.append(time.clock()-start)
                all_TP_train.append(train_N_TP)
                all_FP_train.append(train_N_FP)
                all_TN_train.append(train_N_TN)
                all_FN_train.append(train_N_FN)
                all_count_train.append(train_count)
                all_TP_test.append(test_N_TP)
                all_FP_test.append(test_N_FP)
                all_TN_test.append(test_N_TN)
                all_FN_test.append(test_N_FN)
                all_count_test.append(test_count)


X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 1
Epoch 1/10
1410/1410 [==============================] - 2s 1ms/step - loss: 0.2636 - acc: 0.9333
Epoch 2/10
1410/1410 [==============================] - 0s 275us/step - loss: 0.2541 - acc: 0.9333
Epoch 3/10
1410/1410 [==============================] - 0s 268us/step - loss: 0.2393 - acc: 0.9326
Epoch 4/10
1410/1410 [==============================] - 0s 267us/step - loss: 0.2453 - acc: 0.9333
Epoch 5/10
1410/1410 [==============================] - 0s 275us/step - loss: 0.2247 - acc: 0.9333
Epoch 6/10
1410/1410 [==============================] - 0s 268us/step - loss: 0.2115 - acc: 0.9333
Epoch 7/10
1410/1410 [==============================] - 0s 268us/step - loss: 0.2338 - acc: 0.9312
Epoch 8/10
1410/1410 [==============================] - 0s 275us/step - loss: 0.2226 - acc: 0.9340
Epoch 9/10
1410/1410 [==============================] - 0s 274us/step - loss: 0.2092 - acc: 0.

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars


X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 2
Epoch 1/10
1410/1410 [==============================] - 1s 465us/step - loss: 0.2862 - acc: 0.9262
Epoch 2/10
1410/1410 [==============================] - 0s 268us/step - loss: 0.2435 - acc: 0.9333
Epoch 3/10
1410/1410 [==============================] - 0s 263us/step - loss: 0.2325 - acc: 0.9333
Epoch 4/10
1410/1410 [==============================] - 0s 275us/step - loss: 0.2324 - acc: 0.9340
Epoch 5/10
1410/1410 [==============================] - 0s 268us/step - loss: 0.2238 - acc: 0.9319
Epoch 6/10
1410/1410 [==============================] - 0s 268us/step - loss: 0.2136 - acc: 0.9333
Epoch 7/10
1410/1410 [==============================] - 0s 260us/step - loss: 0.2093 - acc: 0.9333
Epoch 8/10
1410/1410 [==============================] - 0s 252us/step - loss: 0.2124 - acc: 0.9333
Epoch 9/10
1410/1410 [==============================] - 0s 266us/step - loss: 0.2054 - acc: 

1411/1411 [==============================] - 1s 360us/step - loss: 0.2198 - acc: 0.9320
Epoch 7/10
1411/1411 [==============================] - 0s 324us/step - loss: 0.2205 - acc: 0.9341
Epoch 8/10
1411/1411 [==============================] - 1s 383us/step - loss: 0.2106 - acc: 0.9355
Epoch 9/10
1411/1411 [==============================] - 1s 367us/step - loss: 0.1934 - acc: 0.9348
Epoch 10/10
1411/1411 [==============================] - 0s 138us/step
[1317.   94.]
156/156 [==============================] - 0s 13us/step
[146.  10.]
X_train shape :  (1411, 474)
Y_train shape :  (1411, 2)
X_test shape :  (156, 474)
Y_test shape :  (156, 2)
0 0 0 9
Epoch 1/10
1411/1411 [==============================] - 1s 596us/step - loss: 0.2758 - acc: 0.9305
Epoch 2/10
1411/1411 [==============================] - 0s 239us/step - loss: 0.2529 - acc: 0.9334
Epoch 3/10
1411/1411 [==============================] - 0s 239us/step - loss: 0.2468 - acc: 0.9334
Epoch 4/10
1411/1411 [===========================

Epoch 1/10
1410/1410 [==============================] - 1s 771us/step - loss: 0.2650 - acc: 0.9262
Epoch 2/10
1410/1410 [==============================] - 0s 283us/step - loss: 0.2395 - acc: 0.9340
Epoch 3/10
1410/1410 [==============================] - ETA: 0s - loss: 0.2335 - acc: 0.936 - 0s 275us/step - loss: 0.2378 - acc: 0.9340
Epoch 4/10
1410/1410 [==============================] - 0s 277us/step - loss: 0.2325 - acc: 0.9340
Epoch 5/10
1410/1410 [==============================] - 0s 275us/step - loss: 0.2300 - acc: 0.9340
Epoch 6/10
1410/1410 [==============================] - 0s 283us/step - loss: 0.2137 - acc: 0.9312
Epoch 7/10
1410/1410 [==============================] - 0s 283us/step - loss: 0.2107 - acc: 0.9340
Epoch 8/10
1410/1410 [==============================] - 0s 282us/step - loss: 0.2206 - acc: 0.9355
Epoch 9/10
1410/1410 [==============================] - 0s 282us/step - loss: 0.2030 - acc: 0.9355
Epoch 10/10
1410/1410 [==============================] - 0s 225us/step


...

2820/2820 [==============================] - 1s 320us/step - loss: 0.2324 - acc: 0.9340
Epoch 3/10
2820/2820 [==============================] - 1s 319us/step - loss: 0.2205 - acc: 0.9344
Epoch 4/10
2820/2820 [==============================] - 1s 322us/step - loss: 0.2077 - acc: 0.9344
Epoch 5/10
2820/2820 [==============================] - 1s 323us/step - loss: 0.2062 - acc: 0.9348
Epoch 6/10
2820/2820 [==============================] - 1s 319us/step - loss: 0.1884 - acc: 0.9351
Epoch 7/10
2820/2820 [==============================] - 1s 319us/step - loss: 0.1931 - acc: 0.9355
Epoch 8/10
2820/2820 [==============================] - 1s 337us/step - loss: 0.1787 - acc: 0.9404
Epoch 9/10
2820/2820 [==============================] - 1s 318us/step - loss: 0.1735 - acc: 0.9415
Epoch 10/10
1410/1410 [==============================] - 1s 648us/step
[1317.   93.]
157/157 [==============================] - 0s 65us/step
[146.  11.]
X_train shape :  (2820, 474)
Y_train shape :  (2820, 2)
X_test sha

2822/2822 [==============================] - 1s 326us/step - loss: 0.1834 - acc: 0.9373
Epoch 10/10
1411/1411 [==============================] - 1s 711us/step
[1317.   94.]
156/156 [==============================] - 0s 66us/step
[146.  10.]
X_train shape :  (5264, 474)
Y_train shape :  (5264, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
1 1 0 1
Epoch 1/10
5264/5264 [==============================] - 4s 764us/step - loss: 0.5637 - acc: 0.6949
Epoch 2/10
5264/5264 [==============================] - 2s 327us/step - loss: 0.4003 - acc: 0.8167
Epoch 3/10
5264/5264 [==============================] - 2s 326us/step - loss: 0.2566 - acc: 0.8951
Epoch 4/10
5264/5264 [==============================] - 2s 325us/step - loss: 0.1881 - acc: 0.9265
Epoch 5/10
5264/5264 [==============================] - 2s 328us/step - loss: 0.1242 - acc: 0.9567
Epoch 6/10
5264/5264 [==============================] - 2s 325us/step - loss: 0.0736 - acc: 0.9753
Epoch 7/10
5264/5264 [===========================

5238/5238 [==============================] - 2s 332us/step - loss: 0.2383 - acc: 0.9040
Epoch 4/10
5238/5238 [==============================] - 2s 329us/step - loss: 0.1696 - acc: 0.9376
Epoch 5/10
5238/5238 [==============================] - 2s 334us/step - loss: 0.0960 - acc: 0.9683
Epoch 6/10
5238/5238 [==============================] - 2s 332us/step - loss: 0.0589 - acc: 0.9813
Epoch 7/10
5238/5238 [==============================] - 2s 330us/step - loss: 0.0614 - acc: 0.9777
Epoch 8/10
5238/5238 [==============================] - 2s 332us/step - loss: 0.0134 - acc: 0.9969
Epoch 9/10
5238/5238 [==============================] - 2s 367us/step - loss: 0.0452 - acc: 0.9843
Epoch 10/10
1410/1410 [==============================] - 1s 1ms/step
[1317.   93.]
157/157 [==============================] - 0s 65us/step
[146.  11.]
X_train shape :  (5266, 474)
Y_train shape :  (5266, 2)
X_test shape :  (156, 474)
Y_test shape :  (156, 2)
1 1 0 8
Epoch 1/10
5266/5266 [=============================

1410/1410 [==============================] - 1s 859us/step
[1316.   94.]
157/157 [==============================] - 0s 65us/step
[147.  10.]
X_train shape :  (5238, 474)
Y_train shape :  (5238, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
1 1 1 4
Epoch 1/10
5238/5238 [==============================] - 5s 865us/step - loss: 0.6055 - acc: 0.6546
Epoch 2/10
5238/5238 [==============================] - 2s 333us/step - loss: 0.3935 - acc: 0.8230
Epoch 3/10
5238/5238 [==============================] - 2s 334us/step - loss: 0.2560 - acc: 0.8973
Epoch 4/10
5238/5238 [==============================] - 2s 342us/step - loss: 0.1910 - acc: 0.9259
Epoch 5/10
5238/5238 [==============================] - 2s 342us/step - loss: 0.1125 - acc: 0.9643
Epoch 6/10
5238/5238 [==============================] - 2s 343us/step - loss: 0.0856 - acc: 0.9714
Epoch 7/10
5238/5238 [==============================] - 2s 339us/step - loss: 0.0569 - acc: 0.9811
Epoch 8/10
5238/5238 [============================

5266/5266 [==============================] - 2s 303us/step - loss: 0.1755 - acc: 0.9373
Epoch 5/10
5266/5266 [==============================] - 2s 348us/step - loss: 0.1155 - acc: 0.9609
Epoch 6/10
5266/5266 [==============================] - 2s 346us/step - loss: 0.0748 - acc: 0.9774
Epoch 7/10
5266/5266 [==============================] - 2s 347us/step - loss: 0.0745 - acc: 0.9765
Epoch 8/10
5266/5266 [==============================] - 2s 347us/step - loss: 0.0423 - acc: 0.9878
Epoch 9/10
5266/5266 [==============================] - 2s 345us/step - loss: 0.0463 - acc: 0.9839
Epoch 10/10
1411/1411 [==============================] - 1s 953us/step
[1317.   94.]
156/156 [==============================] - 0s 66us/step
[146.  10.]


In [10]:
noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
output = X_train + noise
X_train_all = np.concatenate((X_train, output), axis=0)
print(noise)
print(noise.shape)
print(output.shape)
print(X_train.shape)
print(X_train_all.shape)
for i in range(len(X_train)):
    if (X_train[i]==X_train_all[i]).all()==False:
        print(i)
for i in range(len(X_train)):
    ind = len(X_train)+i
    if (output[i]==X_train_all[ind]).all()==False:
        print(ind)
        

[[-0.00189327  0.00933084  0.00097644 ...  0.0194665   0.00457858
  -0.01221752]
 [ 0.0040473   0.0158048   0.00945874 ... -0.01565949 -0.00438519
   0.00482335]
 [-0.0071294   0.00839675  0.00438763 ... -0.00408452  0.00884706
  -0.008474  ]
 ...
 [-0.01295262 -0.00409624 -0.01216085 ...  0.00048266 -0.00046695
   0.00266308]
 [ 0.01576484 -0.00050172  0.00084574 ... -0.00146055  0.00685842
  -0.01088623]
 [ 0.00634371  0.02780552 -0.00152874 ...  0.01278055  0.00363692
  -0.0129963 ]]
(1411, 474)
(1411, 474)
(1411, 474)
(2822, 474)


In [11]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_TP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
            for iRun in range(nRun):
                print(all_TP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])

for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_count_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_count_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


[1314.   15.]
[79.  2.]
[  15. 1314.]
[ 2. 79.]
[1312.   16.]
[78.  4.]
[  16. 1312.]
[ 4. 78.]
[1316.    8.]
[86.  0.]
[   8. 1316.]
[ 0. 86.]
[1317.    2.]
[91.  0.]
[   2. 1317.]
[ 0. 91.]
[1317.    5.]
[88.  0.]
[   5. 1317.]
[ 0. 88.]
[1.317e+03 1.000e+00]
[92.  0.]
[1.000e+00 1.317e+03]
[ 0. 92.]
[1317.    4.]
[89.  0.]
[   4. 1317.]
[ 0. 89.]
[1306.   30.]
[64. 11.]
[  30. 1306.]
[11. 64.]
[1315.   13.]
[81.  2.]
[  13. 1315.]
[ 2. 81.]
[1317.    3.]
[91.  0.]
[   3. 1317.]
[ 0. 91.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[145.   0.]
[10.  2.]
[  0. 145.]
[ 2. 10.]
[146.   0.]
[10.  1.]
[  0. 146.]
[ 1. 10.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[140.   1.]
[9. 6.]
[  1. 140.]
[6. 9.]
[145.   0.]
[10.  1.]
[  0. 145.]
[ 1. 10.]
[145.   0.]
[10.  1.]
[  0. 145.]
[ 1. 10.]
[1230.   49.]
[45. 86.]
[  49. 1230.]
[86. 45.]
[1316.    2.]
[92.  

In [12]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


5.9584122
4.423525300000001
4.3976018
4.581618200000001
4.752522000000003
4.6730424
5.739695400000002
5.978903899999999
4.273410600000005
4.822227800000007
4.642154500000004
5.044055900000004
5.051789600000006
5.857411099999993
5.047160000000005
5.475431699999987
5.335845899999995
6.124951699999997
5.295413600000018
5.255527000000001
8.156104999999997
8.823544900000002
8.9392493
9.000224599999996
10.476553499999994
7.930603200000007
10.388800000000003
13.29290039999998
9.504860899999983
9.469840199999965
9.445584500000052
9.42550510000001
9.656165399999963
9.637971800000003
9.665225200000009
9.481483700000013
10.778445099999999
10.564336400000002
9.799667699999986
9.90954240000002
10.772242300000016
11.358512099999984
10.511392299999955
10.481567499999983
10.708509899999967
10.614523100000042
10.94587690000003
11.11226899999997
11.506673299999989
11.33142209999994
11.265618700000005
11.3439214
11.496690000000058
12.000528999999915
11.568014299999959
11.38275550000003
12.144622500000082

In [13]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.09706040000000016
0.09219039999999978
0.10495180000000204
0.11907619999999852
0.130416799999999
0.18031900000000434
0.23966459999999756
0.21469590000000238
0.18764409999999998
0.2035920999999945
0.21571299999999383
0.23485030000000506
0.3461400999999995
0.26466800000000035
0.3338285999999897
0.292993299999992
0.30906479999998737
0.31796630000002324
0.334958400000005
0.353810100000004
0.36719420000000014
0.3872669000000144
0.40065679999997883
0.42236370000000534
0.41771069999998645
0.44185730000000945
0.582912599999986
0.5937646999999515
0.48210070000004634
0.5178647000000183
0.5331249999999841
0.5937162000000171
0.5553921000000059
0.5790708999999765
0.7092336999999702
0.6193936999999892
0.7621266999999534
0.6327263000000016
0.6552654000000189
0.6695876999999655
0.8262454000000048
0.6994745999999736
0.7309011000000396
0.7412140999999792
0.902333399999975
0.7720692000000327
0.7906722999999829
0.9679682999999386
0.8754158000000416
0.8465152999999646
0.8579006999999592
0.9709192000000257

In [14]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.011349199999999726
0.008738600000000929
0.008659900000001386
0.00982309999999842
0.00938889999999759
0.011824199999999507
0.016928199999995286
0.010845100000004493
0.012526099999988105
0.009607200000004923
0.009970899999999006
0.009668099999998958
0.012302499999989891
0.009571199999996338
0.012096900000003075
0.010112199999994687
0.00979899999998679
0.009881799999988061
0.010049500000008038
0.010107399999981226
0.009772299999980305
0.00990830000000642
0.010162100000002283
0.010119000000003098
0.009758200000021588
0.010483200000010129
0.012028500000013764
0.01534079999998994
0.01077539999999999
0.010725799999988794
0.010611100000005536
0.009521199999994678
0.01043779999997696
0.010751299999981256
0.01324019999998427
0.01127520000000004
0.015332999999998265
0.010443500000008044
0.011438899999973273
0.01122120000002269
0.013916100000017195
0.010777200000006815
0.011203099999988808
0.012106899999992038
0.014030099999956747
0.011865900000088914
0.010739000000057786
0.01437969999994948
0.0

In [15]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))

#save_path =("./tmp_TP_EP%d") % (1)
#fl=open(save_path, 'w')
#lists=[line+"\n" for line in all_TP_train]
#fl.writelines(lists)